In [3]:
#Imports
from tslearn.clustering import TimeSeriesKMeans
import pandas as pd
import os
import tensorflow as tf
from keras import backend as K

import sys  
sys.path.append("../")  
from utils.modelgenerator import *
from utils.modelhandler import *
from utils.datahandler import *


from sklearn.preprocessing import MinMaxScaler

#Get data 
cwd = os.path.normpath(os.path.dirname(os.path.dirname(os.getcwd())))
df = pd.read_csv(cwd+'/data/2feature_engineering_data/df_with_final_features.csv', index_col='Date') #df = pd.read_csv('user5.csv')
df.index = pd.to_datetime(df.index)
#df = df[['User5', 'temp', 'rhum']]
df.fillna(0, inplace=True)
df.head()

,User1,User10,User11,User12,User13,User14,User15,User16,User17,User18,...,User27_lag_24hrs,User28_lag_24hrs,User29_lag_24hrs,User30_lag_24hrs,User31_lag_24hrs,User32_lag_24hrs,User33_lag_24hrs,User34_lag_24hrs,User35_lag_24hrs,User36_lag_24hrs
Date,,,,,,,,,,,,,,,,,,,,,
2012-07-08 00:00:00,0.200,0.099,0.312,0.152,1.227,0.162,0.089,1.079,0.366,0.063,...,0.040,0.100,0.522,0.058,0.211,0.066,0.627,0.844,0.413,0.117
2012-07-08 01:00:00,0.253,0.060,0.263,0.212,0.132,0.204,0.072,0.714,0.134,0.063,...,0.050,0.131,0.104,0.031,0.271,0.112,0.417,0.350,0.310,0.552
2012-07-08 02:00:00,0.441,0.028,0.257,0.218,0.144,0.193,0.067,0.706,0.127,0.061,...,0.051,0.106,0.163,0.022,0.280,0.121,0.324,0.313,0.285,0.055
2012-07-08 03:00:00,0.220,0.054,0.243,0.225,0.139,0.170,0.045,0.796,0.134,0.061,...,0.111,0.100,0.115,0.032,0.279,0.119,0.306,0.331,0.270,0.067
2012-07-08 04:00:00,0.635,0.060,0.245,0.188,1.157,0.146,0.064,0.755,0.154,0.063,...,0.083,0.094,0.148,0.067,0.184,0.120,0.287,0.275,0.273,0.033


In [4]:
columns_to_keep = []

for column_name in df.columns:
    if column_name.startswith('User') and column_name[4:].isdigit():
        user_number = int(column_name[4:])
        if 1 <= user_number <= 30:
            columns_to_keep.append(column_name)

filtered_df = df[columns_to_keep]
filtered_df.reset_index(inplace=True)
filtered_df

,Date,User1,User10,User11,User12,User13,User14,User15,User16,User17,...,User28,User29,User3,User30,User4,User5,User6,User7,User8,User9
0,2012-07-08 00:00:00,0.200,0.099,0.312,0.152,1.227,0.162,0.089,1.079,0.366,...,0.144,0.418,0.979,0.024,0.092,0.084,0.509,0.106,0.133,0.011
1,2012-07-08 01:00:00,0.253,0.060,0.263,0.212,0.132,0.204,0.072,0.714,0.134,...,0.125,0.065,0.049,0.060,0.090,0.091,0.489,0.067,0.076,0.011
2,2012-07-08 02:00:00,0.441,0.028,0.257,0.218,0.144,0.193,0.067,0.706,0.127,...,0.088,0.048,0.046,0.057,0.091,0.076,0.573,0.066,0.071,0.015
3,2012-07-08 03:00:00,0.220,0.054,0.243,0.225,0.139,0.170,0.045,0.796,0.134,...,0.100,0.049,0.053,0.050,0.098,0.079,0.363,0.084,0.073,0.011
4,2012-07-08 04:00:00,0.635,0.060,0.245,0.188,1.157,0.146,0.064,0.755,0.154,...,0.088,0.066,0.040,0.022,0.096,0.133,0.595,0.129,0.073,0.019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8587,2013-06-30 19:00:00,0.335,0.198,1.362,1.385,0.237,0.331,0.082,0.671,0.418,...,1.381,0.582,0.052,0.350,0.280,2.152,1.018,0.066,0.563,0.089
8588,2013-06-30 20:00:00,0.213,0.873,0.687,1.347,0.282,0.940,0.095,0.507,0.510,...,0.225,0.389,0.050,0.591,0.273,1.993,0.910,0.103,0.104,0.053
8589,2013-06-30 21:00:00,0.190,0.113,0.675,1.615,1.193,0.290,0.115,0.219,0.487,...,0.931,0.749,0.112,0.513,0.253,0.814,1.011,0.076,0.086,0.048
8590,2013-06-30 22:00:00,0.274,0.092,0.747,1.636,0.132,0.265,0.080,0.105,0.197,...,0.119,0.391,0.056,0.168,0.134,0.390,1.063,0.084,0.203,0.072


In [ ]:
"""# Version 1 -------------------------- 4
N_CLUSTERS=4

model = TimeSeriesKMeans(n_clusters=N_CLUSTERS, metric="dtw", max_iter=3, random_state=42)
data_array = np.array(filtered_df.T.drop('Date').values)
model.fit(data_array)
y=model.predict(data_array)
x = filtered_df.Date
print("Clustering results: ", y)"""

In [26]:
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from sklearn.metrics import davies_bouldin_score
import numpy as np

data_array = np.array(filtered_df.T.drop('Date').values)

# Assuming data_array is your time series data with shape (num_samples, num_features)
# You may need to reshape or preprocess your data accordingly

# Rescale the time series data
data_array = TimeSeriesScalerMeanVariance().fit_transform(data_array)

# Reshape the data array to 2D
data_array_2d = data_array.reshape(data_array.shape[0], -1)

# Specify the maximum number of clusters you want to consider
max_clusters = 10

# Initialize variables to store best parameters
best_cluster_count = 0
best_labels = None
best_score = float('inf')  # Initialize with a value that Davies-Bouldin score cannot be

# Iterate over possible cluster counts
for cluster_count in range(2, max_clusters + 1):
    kmeans = TimeSeriesKMeans(n_clusters=cluster_count, verbose=False, random_state=42, metric='euclidean')
    labels = kmeans.fit_predict(data_array)

    # Reshape the labels array to 2D
    labels_2d = labels.reshape(labels.shape[0], -1)

    # Evaluate the clustering performance using Davies-Bouldin index
    score = davies_bouldin_score(data_array_2d, labels_2d)
    print("Score: ", score)

    # Update best parameters if the current score is better
    if score < best_score:
        best_score = score
        best_cluster_count = cluster_count
        best_labels = labels

# Use the best parameters for clustering
final_kmeans = TimeSeriesKMeans(n_clusters=best_cluster_count, verbose=False, random_state=42)
final_labels = final_kmeans.fit_predict(data_array)
print("Final labels: ", final_labels)
print("Best cluster count: ", best_cluster_count)
print("Davies-Bouldin Score: ", best_score)


Score:  2.924705930927995
Score:  3.0801873310111674
Score:  2.711470314718425
Score:  2.313821700008374
Score:  2.041306053916204
Score:  1.941176391482344
Score:  1.5374228780708583
Score:  1.4469305336472624
Score:  1.4561344381909826
Final labels:  [3 1 1 2 1 4 7 2 1 1 2 0 2 2 5 6 1 2 5 1 8 1 1 1 1 1 1 1 1 1]
Best cluster count:  9
Davies-Bouldin Score:  1.4469305336472624


c:\Users\rs1044\Documents\GitHub\Mixture-of-Experts-based-Federated-Learning-for-Energy-Forecasting\.venv\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\rs1044\Documents\GitHub\Mixture-of-Experts-based-Federated-Learning-for-Energy-Forecasting\.venv\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\rs1044\Documents\GitHub\Mixture-of-Experts-based-Federated-Learning-for-Energy-Forecasting\.venv\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

In [21]:
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.metrics import soft_dtw
import numpy as np

data_array = np.array(filtered_df.T.drop('Date').values)

# Assuming data_array is your time series data with shape (num_samples, num_features)
# You may need to reshape or preprocess your data accordingly

# Rescale the time series data
data_array = TimeSeriesScalerMeanVariance().fit_transform(data_array)

# Specify the maximum number of clusters you want to consider
max_clusters = 5

# Initialize variables to store best parameters
best_cluster_count = 0
best_labels = None
best_score = float('inf')  # Initialize with a value that DTW score cannot be

# Iterate over possible cluster counts
for cluster_count in range(4, max_clusters + 1):
    kmeans = TimeSeriesKMeans(n_clusters=cluster_count, metric='softdtw', verbose=False, random_state=42)
    labels = kmeans.fit_predict(data_array)

    # Evaluate the clustering performance using soft DTW
    score = kmeans.inertia_
    print("Score: ", score)

    # Update best parameters if the current score is better
    if score < best_score:
        best_score = score
        best_cluster_count = cluster_count
        best_labels = labels

# Use the best parameters for clustering
final_kmeans = TimeSeriesKMeans(n_clusters=best_cluster_count, metric='softdtw', verbose=False, random_state=42)
final_labels = final_kmeans.fit_predict(data_array)
print("Final labels: ", final_labels)
print("Best cluster count: ", best_cluster_count)
print("Soft DTW Score: ", best_score)


KeyboardInterrupt: 

In [5]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np


data_array = np.array(filtered_df.T.drop('Date').values)

# Assuming data_array is your time series data with shape (num_samples, num_features)
# You may need to reshape or preprocess your data accordingly

# Specify the maximum number of clusters you want to consider
max_clusters = 10

# Initialize variables to store best parameters
best_cluster_count = 0
best_labels = None
best_score = -1  # Initialize with a value that silhouette score cannot be

# Iterate over possible cluster counts
for cluster_count in range(2, max_clusters + 1):
    kmeans = KMeans(n_clusters=cluster_count, random_state=42)
    labels = kmeans.fit_predict(data_array)

    # Evaluate the clustering performance using silhouette score
    score = silhouette_score(data_array, labels)

    # Update best parameters if the current score is better
    if score > best_score:
        best_score = score
        best_cluster_count = cluster_count
        best_labels = labels

# Use the best parameters for clustering
final_kmeans = KMeans(n_clusters=best_cluster_count, random_state=42)
final_labels = final_kmeans.fit_predict(data_array)
print("Final labels: ", final_labels)
print("Best cluster count: ", best_cluster_count)
print("Silhouette Score: ", best_score)

c:\Users\rs1044\Documents\GitHub\Mixture-of-Experts-based-Federated-Learning-for-Energy-Forecasting\.venv\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\rs1044\Documents\GitHub\Mixture-of-Experts-based-Federated-Learning-for-Energy-Forecasting\.venv\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\rs1044\Documents\GitHub\Mixture-of-Experts-based-Federated-Learning-for-Energy-Forecasting\.venv\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress

Final labels:  [0 0 0 0 0 0 0 0 2 0 0 0 3 1 0 0 0 4 6 0 0 0 0 0 0 0 0 0 5 0]
Best cluster count:  7
Silhouette Score:  0.2862933289775697


c:\Users\rs1044\Documents\GitHub\Mixture-of-Experts-based-Federated-Learning-for-Energy-Forecasting\.venv\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
